# Clasificador para estimar el espaciamiento espectral

- [x] Evolucionar la red neuronal a un clasificador basado en los mejores parámetros y probar con diferente número de clases (de 2 a 5 clases, como Alejandro).
- [x] Añadir el caso sin tomar la OSNR como característica en clasificación.
- [x] Cambiar marcadores para que sean distinguibles FCM vs. GKM (Ser consistente para train y test).
- [x] Graficar matrices de confusión.
- [x] Añadir el caso sin tomar la OSNR como característica en regresión.

In [ ]:
# Imports
import sofa
import polars as pl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow.keras as ker

from sklearn.metrics import (accuracy_score, f1_score, multilabel_confusion_matrix,
                             ConfusionMatrixDisplay)
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import models, regularizers, Sequential, utils
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from time import time

## Funciones globales

In [ ]:
def calc_once(varname, fn, args):
    """ Calculate a variable only once. """
    if varname not in globals():
        return fn(**args)
    return eval(varname)

In [ ]:
def plot_loss(score, end=500):
    """ Plot loss evolution for each k-fold. """
    for k, loss in enumerate(score["loss"]):
        loss_length = len(loss.history["loss"])
        loss_values = loss.history["loss"][:end if loss_length > end else loss_length]
        epoch_values = range(end if loss_length > end else loss_length)
        plt.plot(epoch_values, loss_values, label=f"k = {k+1}")

def plot_losses(scores, scenario, cmp_fn, cmp_values=[], end=500, based_on_index=False):
    """ Plot loss graphics for each scenario. """
    # Handle default cmp_values
    if len(cmp_values) == 0:
        cmp_values = np.zeros(len(scores))
        
    fig_loss = plt.figure(figsize=(16, 3*len(scores)), layout="constrained")
    fig_loss.suptitle(f"{scenario} loss history", size="x-large")
    
    for index, (score, cmps) in enumerate(zip(scores, cmp_values)):
        plt.subplot(len(scores)//2, 2, index + 1)
        plot_loss(score, end=end)
        
        plt.title(cmp_fn(cmps) if not based_on_index else cmp_fn(index))
        
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        
        # Transparent white box black edge legend
        legend = plt.legend(loc="upper right", edgecolor="black")
        legend.get_frame().set_alpha(None)
        legend.get_frame().set_facecolor((1, 1, 1, 0.01))
        
        plt.grid(True)

def plot_scores(scores, x_values, scenario, score_names, data_type,
                label, xlabel, markers=[], colors=[], based_on_index=False,
                log=False, multiple_points=False, plot_train=True):
    fig_scores = plt.figure(figsize=(16, 6), layout="constrained")
    fig_scores.suptitle(f"{scenario} scores")
    i = 0
    for sn in score_names:
        ax = plt.subplot(1, len(score_names), i+1)
        for dt in data_type:
            if not plot_train and dt == "train":
                continue
            points = [np.average(score[sn][dt]) for score in scores]
            if not multiple_points:
                label_value = ""
                plt.scatter(x_values, points, marker=markers[0 if dt == "train" else 1],
                            label=f"{dt.title()} {label(i)}", s=100)
            else:
                label_value = ""
                points1 = points[::2]
                points2 = points[1::2]
                plt.scatter(x_values, points1, marker=markers[0 if dt == "train" else 1],
                            color=colors[0 if dt == "train" else 1],
                            label=f"{dt.title()} {label(0)}", s=100)
                plt.scatter(x_values, points2, marker=markers[2 if dt == "train" else 3],
                            color=colors[2 if dt == "train" else 3],
                            label=f"{dt.title()} {label(1)}", s=100)
            title = ""
            if sn == "r2":
                title = "R²"
            elif sn == "acc":
                title = "Accuracy"
            else:
                title = sn.upper()
            plt.title(title)
                
        plt.xlabel(xlabel)
        # Adjust logarithmic scale if requested
        if log:
            plt.xscale("log", base=2)
            
        # Make integer xticks if matches}
        if type(x_values[0]) == np.int64:
            ax.set_xticks(x_values)
            
        if type(x_values[0]) == str:
            ax.set_xticks(range(len(x_values)), x_values)
            
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Transparent white box black edge legend
        legend = plt.legend(loc="center left", bbox_to_anchor=(1, 0.5),
                            edgecolor="black")
        legend.get_frame().set_alpha(None)
        legend.get_frame().set_facecolor((1, 1, 1, 0.01))

        plt.grid(True)
        i += 1
    plt.show()

### Funciones para clasificación

In [ ]:
def classificator(df, interval_lst, column_name):
    """Transforms a dataframe's column into classes"""
    array = df[column_name].to_numpy()
    indexes_lst = []
    for i, interval in enumerate(interval_lst):
        lower_limit, upper_limit = interval
        indexes_lst.append(np.intersect1d(np.where(lower_limit < array), np.where(array <= upper_limit)))
    
    classfull = df[column_name]
    for index, indexes in enumerate(indexes_lst):
        classfull[indexes] = index

    df_classfull = df.clone()
    df_classfull.replace(column_name, classfull)
    
    return df_classfull


def classifier_model(input_dim, layers_props_lst, classes_n, loss_fn):
    """ Compile a sequential model for classification purposes. """
    model = ker.Sequential()
    # Hidden layers
    for i, layer_props in enumerate(layers_props_lst):
        if i == 0:
            model.add(ker.layers.Dense(**layer_props, input_dim=input_dim))
        else:
            model.add(ker.layers.Dense(**layer_props))
    # Classifier
    model.add(ker.layers.Dense(units=classes_n, activation="softmax"))

    model.compile(loss=loss_fn, optimizer="adam")
    return model    


def classification_crossvalidation(X, y, n_splits, layer_props, classes_n, loss_fn, callbacks):
    """ Crossvalidation of a classification network. """
    # Store initial time
    t0 = time()

    # Scores dict
    scores = {}
    scores["loss"] = []
    scores["acc"] = {"train": [], "test": []}
    scores["f1"] = {"train": [], "test": []}
    scores["cm"] = {"train": [], "test": []}
    
    # K-fold crossvalidation
    kf = KFold(n_splits=n_splits, shuffle=True)

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Input variables standarizer
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test_kf = sc.transform(X_test)
        
        model = classifier_model(X_train.shape[1], layer_props, classes_n, loss_fn)
        
        # Save test scalar loss
        if callbacks:
            loss = model.fit(
                X_train, y_train, epochs=5000, batch_size=64, verbose=0, callbacks=callbacks
            )
        else:
            loss = model.fit(X_train, y_train, epochs=5000, batch_size=64, verbose=0)
        print(f"Needed iterations: {len(loss.history['loss'])}")
        
        # Predict using train values
        fuzzy_predictions_train = model.predict(X_train)
        # Predict using test values
        fuzzy_predictions_test = model.predict(X_test_kf)
        
        # Assign class based on higher probability in membership vector
        predictions_train = np.array([np.argmax(fuzzy_prediction) for fuzzy_prediction in fuzzy_predictions_train])
        predictions_test = np.array([np.argmax(fuzzy_prediction) for fuzzy_prediction in fuzzy_predictions_test])

        # Dataframe for better visualization
        train_data_train = pl.DataFrame(
            {"ICI": [y_train], "Predicted ICI": [predictions_train]}
        )
        train_data_test = pl.DataFrame(
            {"ICI": [y_test], "Predicted ICI": [predictions_test]}
        )

        # Accuracy
        acc_score_train = accuracy_score(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"]
        )
        acc_score_test = accuracy_score(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"]
        )

        # F1
        f1_score_train = f1_score(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"],
            average="micro"
        )
        f1_score_test = f1_score(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"],
            average="micro"
        )
         
        # Confusion Matrix
        cm_score_train = multilabel_confusion_matrix(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"]
        )
        cm_score_test = multilabel_confusion_matrix(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"]
        )

        # Append to lists
        scores["loss"].append(loss)
        scores["acc"]["train"].append(acc_score_train)
        scores["acc"]["test"].append(acc_score_test)
        scores["f1"]["train"].append(f1_score_train)
        scores["f1"]["test"].append(f1_score_test)
        scores["cm"]["train"].append(cm_score_train)
        scores["cm"]["test"].append(cm_score_test)
        
    print(f"Time elapsed: {(time() - t0)/60:.2f} minutes")

    return scores


def test_classification_model(data, n_splits, max_neurons, activations, classes_n,
                              use_osnr=True, loss_fn="sparse_categorical_crossentropy"):
    """ Test a spectral overlapping classification model with given parameters. """
    # Set variable number
    var_n = 17 if use_osnr else 16
    
    # Split variables
    # Variables
    X = np.array(data[:, 0:var_n]).T
    # Tags
    y = np.array(data[:, 19:20]).T
    
    # Layer properties
    layer_props = [
        {"units": max_neurons // (2**i), "activation": activation}
        for i, activation in enumerate(activations)
    ]
    print(layer_props)
    callbacks = [
        EarlyStopping(monitor="loss", patience=30, mode="min", restore_best_weights=True)
    ]
    
    return classification_crossvalidation(X, y, n_splits, layer_props, classes_n, loss_fn, callbacks)

def plot_classes_scores(scores, scenario):
    score_names = ["acc"]
    data_type = ["train", "test"]
    markers = ["o", "D", "o", "D"]
    colors = ["dodgerblue", "dodgerblue", "red", "red"]
    
    # Plot loss
    plot_losses(scores, scenario, 
                lambda index: f"{'FCM' if index%2==0 else 'GKM'} {'B2B' if index<2 else 'optical fiber at ' + ('0' if index<4 else '9') + ' dBm'}",
                based_on_index=True)
    
    # Plot scores
    plot_scores(scores, ["B2B", "0 dBm", "9 dBm"], scenario, score_names,
                data_type, label=lambda index: f"{'FCM' if index%2==0 else 'GKM'}",
                xlabel="Scenario", markers=markers, colors=colors, multiple_points=True,
                based_on_index=True)
    plt.show()
    
def plot_cm(scores, interval_lst):
    CM = np.array(scores.get("cm").get("test"))
    for n, interval in enumerate(interval_lst):
        result = np.zeros(CM[0][0].shape)
        for cm in CM:
            result = np.add(result, cm[n])
        result /= np.sum(result)
        disp = ConfusionMatrixDisplay(confusion_matrix=result, display_labels=["Positive", "Negative"])
        disp.plot(colorbar=False)
        lower_limit, upper_limit = interval 
        plt.title(f"Confusion matrix for class from {lower_limit} GHz up to {upper_limit} GHz")
        plt.show()

## Restaurar variables

#### Clasificación con 2 clases

In [ ]:
%store -r scores_2classes_FCM_B2B
%store -r scores_2classes_GKM_B2B

%store -r scores_2classes_FCM_fiber0
%store -r scores_2classes_GKM_fiber0

%store -r scores_2classes_FCM_fiber9
%store -r scores_2classes_GKM_fiber9

%store -r scores_2classes_woOSNR_FCM_B2B
%store -r scores_2classes_woOSNR_GKM_B2B

%store -r scores_2classes_woOSNR_FCM_fiber0
%store -r scores_2classes_woOSNR_GKM_fiber0

%store -r scores_2classes_woOSNR_FCM_fiber9
%store -r scores_2classes_woOSNR_GKM_fiber9

#### Clasificación con 3 clases

In [ ]:
%store -r scores_3classes_FCM_B2B
%store -r scores_3classes_GKM_B2B

%store -r scores_3classes_FCM_fiber0
%store -r scores_3classes_GKM_fiber0

%store -r scores_3classes_FCM_fiber9
%store -r scores_3classes_GKM_fiber9

%store -r scores_3classes_woOSNR_FCM_B2B
%store -r scores_3classes_woOSNR_GKM_B2B

%store -r scores_3classes_woOSNR_FCM_fiber0
%store -r scores_3classes_woOSNR_GKM_fiber0

%store -r scores_3classes_woOSNR_FCM_fiber9
%store -r scores_3classes_woOSNR_GKM_fiber9

#### Clasificación con 4 clases

In [ ]:
%store -r scores_4classes_FCM_B2B
%store -r scores_4classes_GKM_B2B

%store -r scores_4classes_FCM_fiber0
%store -r scores_4classes_GKM_fiber0

%store -r scores_4classes_FCM_fiber9
%store -r scores_4classes_GKM_fiber9

%store -r scores_4classes_woOSNR_FCM_B2B
%store -r scores_4classes_woOSNR_GKM_B2B

%store -r scores_4classes_woOSNR_FCM_fiber0
%store -r scores_4classes_woOSNR_GKM_fiber0

%store -r scores_4classes_woOSNR_FCM_fiber9
%store -r scores_4classes_woOSNR_GKM_fiber9

#### Clasificación con 5 clases

In [ ]:
%store -r scores_5classes_FCM_B2B
%store -r scores_5classes_GKM_B2B

%store -r scores_5classes_FCM_fiber0
%store -r scores_5classes_GKM_fiber0

%store -r scores_5classes_FCM_fiber9
%store -r scores_5classes_GKM_fiber9

%store -r scores_5classes_woOSNR_FCM_B2B
%store -r scores_5classes_woOSNR_GKM_B2B

%store -r scores_5classes_woOSNR_FCM_fiber0
%store -r scores_5classes_woOSNR_GKM_fiber0

%store -r scores_5classes_woOSNR_FCM_fiber9
%store -r scores_5classes_woOSNR_GKM_fiber9

## Cargar los datos

### FCM

In [ ]:
data_file_FCM = "../Databases/CaractFCM.csv"
df_FCM = pl.read_csv(data_file_FCM, dtypes=[pl.Float64], has_header=False)

### GKM

In [ ]:
data_file_GKM = "../Databases/CaractGKM.csv"
df_GKM = pl.read_csv(data_file_GKM, dtypes=[pl.Float64], infer_schema_length = 10000, has_header=False)

# Clasificador

## 2 clases

### FCM

In [ ]:
interval_lst = [(0, 35), (35, 50)]
df_2classes_FCM = classificator(df_FCM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_2classes_FCM_B2B = df_2classes_FCM[0:987, :]
# 0dBm optical fiber
df_2classes_FCM_fiber0 = df_2classes_FCM[988:1520, :]
# 9dBm optical fiber
df_2classes_FCM_fiber9 = df_2classes_FCM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_2classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_2classes_FCM_B2B = calc_once("scores_2classes_FCM_B2B", test_classification_model, args)
plot_cm(scores_2classes_FCM_B2B, interval_lst)

In [ ]:
args = {"data": df_2classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_2classes_woOSNR_FCM_B2B = calc_once("scores_2classes_woOSNR_FCM_B2B", test_classification_model, args)
plot_cm(scores_2classes_woOSNR_FCM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
args = {"data": df_2classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_2classes_FCM_fiber0 = calc_once("scores_2classes_FCM_fiber0", test_classification_model, args)
plot_cm(scores_2classes_FCM_fiber0, interval_lst)

In [ ]:
args = {"data": df_2classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_2classes_woOSNR_FCM_fiber0 = calc_once("scores_2classes_woOSNR_FCM_fiber0", test_classification_model, args)
plot_cm(scores_2classes_woOSNR_FCM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_2classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_2classes_FCM_fiber9 = calc_once("scores_2classes_FCM_fiber9", test_classification_model, args)
plot_cm(scores_2classes_FCM_fiber9, interval_lst)

In [ ]:
args = {"data": df_2classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_2classes_woOSNR_FCM_fiber9 = calc_once("scores_2classes_woOSNR_FCM_fiber9", test_classification_model, args)
plot_cm(scores_2classes_woOSNR_FCM_fiber9, interval_lst)

### GKM

In [ ]:
interval_lst = [(0, 35), (35, 50)]
df_2classes_GKM = classificator(df_GKM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_2classes_GKM_B2B = df_2classes_GKM[0:987, :]
# 0dBm optical fiber
df_2classes_GKM_fiber0 = df_2classes_GKM[988:1520, :]
# 9dBm optical fiber
df_2classes_GKM_fiber9 = df_2classes_GKM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_2classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_2classes_GKM_B2B = calc_once("scores_2classes_GKM_B2B", test_classification_model, args)
plot_cm(scores_2classes_GKM_B2B, interval_lst)

In [ ]:
args = {"data": df_2classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_2classes_woOSNR_GKM_B2B = calc_once("scores_2classes_woOSNR_GKM_B2B", test_classification_model, args)
plot_cm(scores_2classes_woOSNR_GKM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
args = {"data": df_2classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_2classes_GKM_fiber0 = calc_once("scores_2classes_GKM_fiber0", test_classification_model, args)
plot_cm(scores_2classes_GKM_fiber0, interval_lst)

In [ ]:
args = {"data": df_2classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_2classes_woOSNR_GKM_fiber0 = calc_once("scores_2classes_woOSNR_GKM_fiber0", test_classification_model, args)
plot_cm(scores_2classes_woOSNR_GKM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_2classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_2classes_GKM_fiber9 = calc_once("scores_2classes_GKM_fiber9", test_classification_model, args)
plot_cm(scores_2classes_GKM_fiber9, interval_lst)

In [ ]:
args = {"data": df_2classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_2classes_woOSNR_GKM_fiber9 = calc_once("scores_2classes_woOSNR_GKM_fiber9", test_classification_model, args)
plot_cm(scores_2classes_woOSNR_GKM_fiber9, interval_lst)

In [ ]:
%store scores_2classes_FCM_B2B
%store scores_2classes_GKM_B2B
%store scores_2classes_FCM_fiber0
%store scores_2classes_GKM_fiber0
%store scores_2classes_FCM_fiber9
%store scores_2classes_GKM_fiber9

%store scores_2classes_woOSNR_FCM_B2B
%store scores_2classes_woOSNR_GKM_B2B
%store scores_2classes_woOSNR_FCM_fiber0
%store scores_2classes_woOSNR_GKM_fiber0
%store scores_2classes_woOSNR_FCM_fiber9
%store scores_2classes_woOSNR_GKM_fiber9

### Gráficas

In [ ]:
scores = [scores_2classes_FCM_B2B, scores_2classes_GKM_B2B, scores_2classes_FCM_fiber0, scores_2classes_GKM_fiber0, scores_2classes_FCM_fiber9, scores_2classes_GKM_fiber0]
plot_classes_scores(scores, "2 classes evaluation")

## 3 clases

### FCM

In [ ]:
interval_lst = [(0, 32), (32, 35), (35, 50)]
df_3classes_FCM = classificator(df_FCM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_3classes_FCM_B2B = df_3classes_FCM[0:987, :]
# 0dBm optical fiber
df_3classes_FCM_fiber0 = df_3classes_FCM[988:1520, :]
# 9dBm optical fiber
df_3classes_FCM_fiber9 = df_3classes_FCM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_3classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_3classes_FCM_B2B = calc_once("scores_3classes_FCM_B2B", test_classification_model, args)
plot_cm(scores_3classes_FCM_B2B, interval_lst)

In [ ]:
args = {"data": df_3classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_3classes_woOSNR_FCM_B2B = calc_once("scores_3classes_woOSNR_FCM_B2B", test_classification_model, args)
plot_cm(scores_3classes_woOSNR_FCM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
args = {"data": df_3classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_3classes_FCM_fiber0 = calc_once("scores_3classes_FCM_fiber0", test_classification_model, args)
plot_cm(scores_3classes_FCM_fiber0, interval_lst)

In [ ]:
args = {"data": df_3classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_3classes_woOSNR_FCM_fiber0 = calc_once("scores_3classes_woOSNR_FCM_fiber0", test_classification_model, args)
plot_cm(scores_3classes_woOSNR_FCM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_3classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_3classes_FCM_fiber9 = calc_once("scores_3classes_FCM_fiber9", test_classification_model, args)
plot_cm(scores_3classes_FCM_fiber9, interval_lst)

In [ ]:
args = {"data": df_3classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_3classes_woOSNR_FCM_fiber9 = calc_once("scores_3classes_woOSNR_FCM_fiber9", test_classification_model, args)
plot_cm(scores_3classes_woOSNR_FCM_fiber9, interval_lst)

### GKM

In [ ]:
interval_lst = [(0, 32), (32, 35), (35, 50)]
df_3classes_GKM = classificator(df_GKM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_3classes_GKM_B2B = df_3classes_GKM[0:987, :]
# 0dBm optical fiber
df_3classes_GKM_fiber0 = df_3classes_GKM[988:1520, :]
# 9dBm optical fiber
df_3classes_GKM_fiber9 = df_3classes_GKM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_3classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_3classes_GKM_B2B = calc_once("scores_3classes_GKM_B2B", test_classification_model, args)
plot_cm(scores_3classes_GKM_B2B, interval_lst)

In [ ]:
args = {"data": df_3classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_3classes_woOSNR_GKM_B2B = calc_once("scores_3classes_woOSNR_GKM_B2B", test_classification_model, args)
plot_cm(scores_3classes_woOSNR_GKM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
args = {"data": df_3classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_3classes_GKM_fiber0 = calc_once("scores_3classes_GKM_fiber0", test_classification_model, args)
plot_cm(scores_3classes_GKM_fiber0, interval_lst)

In [ ]:
args = {"data": df_3classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_3classes_woOSNR_GKM_fiber0 = calc_once("scores_3classes_woOSNR_GKM_fiber0", test_classification_model, args)
plot_cm(scores_3classes_woOSNR_GKM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_3classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_3classes_GKM_fiber9 = calc_once("scores_3classes_GKM_fiber9", test_classification_model, args)
plot_cm(scores_3classes_GKM_fiber9, interval_lst)

In [ ]:
args = {"data": df_3classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_3classes_woOSNR_GKM_fiber9 = calc_once("scores_3classes_woOSNR_GKM_fiber9", test_classification_model, args)
plot_cm(scores_3classes_woOSNR_GKM_fiber9, interval_lst)

In [ ]:
%store scores_3classes_FCM_B2B
%store scores_3classes_GKM_B2B
%store scores_3classes_FCM_fiber0
%store scores_3classes_GKM_fiber0
%store scores_3classes_FCM_fiber9
%store scores_3classes_GKM_fiber9

%store scores_3classes_woOSNR_FCM_B2B
%store scores_3classes_woOSNR_GKM_B2B
%store scores_3classes_woOSNR_FCM_fiber0
%store scores_3classes_woOSNR_GKM_fiber0
%store scores_3classes_woOSNR_FCM_fiber9
%store scores_3classes_woOSNR_GKM_fiber9

### Gráficas

In [ ]:
scores = [scores_3classes_FCM_B2B, scores_3classes_GKM_B2B, scores_3classes_FCM_fiber0, scores_3classes_GKM_fiber0, scores_3classes_FCM_fiber9, scores_3classes_GKM_fiber0]
plot_classes_scores(scores, "3 classes evaluation")

## 4 clases

### FCM

In [ ]:
interval_lst = [(0, 31.5), (31.5, 33.5), (33.5, 35), (35, 50)]
df_4classes_FCM = classificator(df_FCM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_4classes_FCM_B2B = df_4classes_FCM[0:987, :]
# 0dBm optical fiber
df_4classes_FCM_fiber0 = df_4classes_FCM[988:1520, :]
# 9dBm optical fiber
df_4classes_FCM_fiber9 = df_4classes_FCM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_4classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_4classes_FCM_B2B = calc_once("scores_4classes_FCM_B2B", test_classification_model, args)
plot_cm(scores_4classes_FCM_B2B, interval_lst)

In [ ]:
args = {"data": df_4classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_4classes_woOSNR_FCM_B2B = calc_once("scores_4classes_woOSNR_FCM_B2B", test_classification_model, args)
plot_cm(scores_4classes_woOSNR_FCM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
args = {"data": df_4classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_4classes_FCM_fiber0 = calc_once("scores_4classes_FCM_fiber0", test_classification_model, args)
plot_cm(scores_4classes_FCM_fiber0, interval_lst)

In [ ]:
args = {"data": df_4classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_4classes_woOSNR_FCM_fiber0 = calc_once("scores_4classes_woOSNR_FCM_fiber0", test_classification_model, args)
plot_cm(scores_4classes_woOSNR_FCM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_4classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_4classes_FCM_fiber9 = calc_once("scores_4classes_FCM_fiber9", test_classification_model, args)
plot_cm(scores_4classes_FCM_fiber9, interval_lst)

In [ ]:
args = {"data": df_4classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_4classes_woOSNR_FCM_fiber9 = calc_once("scores_4classes_woOSNR_FCM_fiber9", test_classification_model, args)
plot_cm(scores_4classes_woOSNR_FCM_fiber9, interval_lst)

### GKM

In [ ]:
interval_lst = [(0, 31.5), (31.5, 33.5), (33.5, 35), (35, 50)]
df_4classes_GKM = classificator(df_GKM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_4classes_GKM_B2B = df_4classes_GKM[0:987, :]
# 0dBm optical fiber
df_4classes_GKM_fiber0 = df_4classes_GKM[988:1520, :]
# 9dBm optical fiber
df_4classes_GKM_fiber9 = df_4classes_GKM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_4classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_4classes_GKM_B2B = calc_once("scores_4classes_GKM_B2B", test_classification_model, args)
plot_cm(scores_4classes_GKM_B2B, interval_lst)

In [ ]:
args = {"data": df_4classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_4classes_woOSNR_GKM_B2B = calc_once("scores_4classes_woOSNR_GKM_B2B", test_classification_model, args)
plot_cm(scores_4classes_woOSNR_GKM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
args = {"data": df_4classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_4classes_GKM_fiber0 = calc_once("scores_4classes_GKM_fiber0", test_classification_model, args)
plot_cm(scores_4classes_GKM_fiber0, interval_lst)

In [ ]:
args = {"data": df_4classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_4classes_woOSNR_GKM_fiber0 = calc_once("scores_4classes_woOSNR_GKM_fiber0", test_classification_model, args)
plot_cm(scores_4classes_woOSNR_GKM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_4classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_4classes_GKM_fiber9 = calc_once("scores_4classes_GKM_fiber9", test_classification_model, args)
plot_cm(scores_4classes_GKM_fiber9, interval_lst)

In [ ]:
args = {"data": df_4classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_4classes_woOSNR_GKM_fiber9 = calc_once("scores_4classes_woOSNR_GKM_fiber9", test_classification_model, args)
plot_cm(scores_4classes_woOSNR_GKM_fiber9, interval_lst)

In [ ]:
%store scores_4classes_FCM_B2B
%store scores_4classes_GKM_B2B
%store scores_4classes_FCM_fiber0
%store scores_4classes_GKM_fiber0
%store scores_4classes_FCM_fiber9
%store scores_4classes_GKM_fiber9

%store scores_4classes_woOSNR_FCM_B2B
%store scores_4classes_woOSNR_GKM_B2B
%store scores_4classes_woOSNR_FCM_fiber0
%store scores_4classes_woOSNR_GKM_fiber0
%store scores_4classes_woOSNR_FCM_fiber9
%store scores_4classes_woOSNR_GKM_fiber9

### Gráficas

In [ ]:
scores = [scores_4classes_FCM_B2B, scores_4classes_GKM_B2B, scores_4classes_FCM_fiber0, scores_4classes_GKM_fiber0, scores_4classes_FCM_fiber9, scores_4classes_GKM_fiber0]
plot_classes_scores(scores, "4 classes evaluation")

## 5 clases

### FCM

In [ ]:
interval_lst = [(0, 31.5), (31.5, 33.5), (33.5, 35), (35, 37.5), (37.5, 50)]
df_5classes_FCM = classificator(df_FCM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_5classes_FCM_B2B = df_5classes_FCM[0:987, :]
# 0dBm optical fiber
df_5classes_FCM_fiber0 = df_5classes_FCM[988:1520, :]
# 9dBm optical fiber
df_5classes_FCM_fiber9 = df_5classes_FCM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_5classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_5classes_FCM_B2B = calc_once("scores_5classes_FCM_B2B", test_classification_model, args)
plot_cm(scores_5classes_FCM_B2B, interval_lst)

In [ ]:
args = {"data": df_5classes_FCM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_5classes_woOSNR_FCM_B2B = calc_once("scores_5classes_woOSNR_FCM_B2B", test_classification_model, args)
plot_cm(scores_5classes_woOSNR_FCM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
bargs = {"data": df_5classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_5classes_FCM_fiber0 = calc_once("scores_5classes_FCM_fiber0", test_classification_model, args)
plot_cm(scores_5classes_FCM_fiber0, interval_lst)

In [ ]:
args = {"data": df_5classes_FCM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_5classes_woOSNR_FCM_fiber0 = calc_once("scores_5classes_woOSNR_FCM_fiber0", test_classification_model, args)
plot_cm(scores_5classes_woOSNR_FCM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_5classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_5classes_FCM_fiber9 = calc_once("scores_5classes_FCM_fiber9", test_classification_model, args)
plot_cm(scores_5classes_FCM_fiber9, interval_lst)

In [ ]:
args = {"data": df_5classes_FCM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_5classes_woOSNR_FCM_fiber9 = calc_once("scores_5classes_woOSNR_FCM_fiber9", test_classification_model, args)
plot_cm(scores_5classes_woOSNR_FCM_fiber9, interval_lst)

### GKM

In [ ]:
interval_lst = [(0, 31.5), (31.5, 33.5), (33.5, 35), (35, 37.5), (37.5, 50)]
df_5classes_GKM = classificator(df_GKM, interval_lst, "column_20")

# Scenarios split
# Back-to-back (B2B)
df_5classes_GKM_B2B = df_5classes_GKM[0:987, :]
# 0dBm optical fiber
df_5classes_GKM_fiber0 = df_5classes_GKM[988:1520, :]
# 9dBm optical fiber
df_5classes_GKM_fiber9 = df_5classes_GKM[1521:, :]

#### B2B

In [ ]:
args = {"data": df_5classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_5classes_GKM_B2B = calc_once("scores_5classes_GKM_B2B", test_classification_model, args)
plot_cm(scores_5classes_GKM_B2B, interval_lst)

In [ ]:
args = {"data": df_5classes_GKM_B2B, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_5classes_woOSNR_GKM_B2B = calc_once("scores_5classes_woOSNR_GKM_B2B", test_classification_model, args)
plot_cm(scores_5classes_woOSNR_GKM_B2B, interval_lst)

#### Fibra óptica a 0dBm

In [ ]:
args = {"data": df_5classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_5classes_GKM_fiber0 = calc_once("scores_5classes_GKM_fiber0", test_classification_model, args)
plot_cm(scores_5classes_GKM_fiber0, interval_lst)

In [ ]:
args = {"data": df_5classes_GKM_fiber0, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_5classes_woOSNR_GKM_fiber0 = calc_once("scores_5classes_woOSNR_GKM_fiber0", test_classification_model, args)
plot_cm(scores_5classes_woOSNR_GKM_fiber0, interval_lst)

#### Fibra óptica a 9dBm

In [ ]:
args = {"data": df_5classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy"}
scores_5classes_GKM_fiber9 = calc_once("scores_5classes_GKM_fiber9", test_classification_model, args)
plot_cm(scores_5classes_GKM_fiber9, interval_lst)

In [ ]:
args = {"data": df_5classes_GKM_fiber9, "n_splits": 5, "max_neurons": 256, "activations": ["relu", "tanh", "sigmoid"], "classes_n": len(interval_lst), "loss_fn": "sparse_categorical_crossentropy", "use_osnr": False}
scores_5classes_woOSNR_GKM_fiber9 = calc_once("scores_5classes_woOSNR_GKM_fiber9", test_classification_model, args)
plot_cm(scores_5classes_woOSNR_GKM_fiber9, interval_lst)

In [ ]:
%store scores_5classes_FCM_B2B
%store scores_5classes_GKM_B2B
%store scores_5classes_FCM_fiber0
%store scores_5classes_GKM_fiber0
%store scores_5classes_FCM_fiber9
%store scores_5classes_GKM_fiber9

%store scores_5classes_woOSNR_FCM_B2B
%store scores_5classes_woOSNR_GKM_B2B
%store scores_5classes_woOSNR_FCM_fiber0
%store scores_5classes_woOSNR_GKM_fiber0
%store scores_5classes_woOSNR_FCM_fiber9
%store scores_5classes_woOSNR_GKM_fiber9

### Gráficas

In [ ]:
scores = [scores_5classes_FCM_B2B, scores_5classes_GKM_B2B, scores_5classes_FCM_fiber0, scores_5classes_GKM_fiber0, scores_5classes_FCM_fiber9, scores_5classes_GKM_fiber0]
plot_classes_scores(scores, "5 classes evaluation")

## Gráfica final

In [ ]:
algorithms = ["FCM", "GKM"]
classes_n = [2, 3, 4, 5]
scenarios = ["B2B", "fiber0", "fiber9"]
OSNR = ["", "_woOSNR"]
classes_scores = {f"scores{osnr}_{algo}_{scenario}":
                  [np.average(
                      eval(f"scores_{n}classes{osnr}_{algo}_{scenario}").get(
                          "acc"
                      ).get(
                          "test"
                      )
                  ) for n in classes_n]
                  for algo in algorithms for osnr in OSNR for scenario in scenarios
                 }
# Markers for algorithm 
markers = {"FCM": "^", "GKM": "s"}

# Colors for scenario (blue for B2B, orange for 0dBm, red for 9dBm)
colors = {"B2B": ["blue", "magenta"], "fiber0": ["orange", "green"], "fiber9": ["red", "cyan"]}

plt.figure(figsize=(8, 6), layout="constrained")
ax = plt.subplot(1, 1, 1)
for algo in algorithms:
    for scenario in scenarios:
        for osnr in OSNR:
            scenario_label = "Fiber @ 0 dBm" if scenario == "fiber0" else "Fiber @ 9 dBm" if scenario != "B2B" else scenario
            OSNR_label = "+ OSNR" if osnr == "" else ""
            label = f"{algo} {scenario_label} {OSNR_label}"
            color = colors.get(scenario)[0 if OSNR_label == "" else 1]
            marker = markers.get(algo)
            markeredgecolor = "black" if algo == "FCM" else None
            plt.plot(classes_n, classes_scores.get(f"scores{osnr}_{algo}_{scenario}"),
                     color=color, marker=marker, linestyle="--",
                     markeredgecolor=markeredgecolor,
                     label=f"{label}")
plt.xlabel("Number of classes")
ax.set_xticks(classes_n)
plt.ylabel("Accuracy")
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Transparent white box black edge legend
legend = plt.legend(loc="center left", bbox_to_anchor=(1, 0.5),
                    edgecolor="black")
legend.get_frame().set_alpha(None)
legend.get_frame().set_facecolor((1, 1, 1, 0.01))
plt.grid(True)

plt.savefig("classes.svg", format="svg", transparent=True, bbox_inches="tight")

## Tabla

In [ ]:
scores = [scores_2classes_FCM_B2B, scores_3classes_FCM_B2B, scores_4classes_FCM_B2B, scores_5classes_FCM_B2B]
scores_classes_FCM_B2B = [float(np.average(score_classes_FCM_B2B.get("acc").get("test"))) for score_classes_FCM_B2B in scores]

scores = [scores_2classes_FCM_fiber0, scores_3classes_FCM_fiber0, scores_4classes_FCM_fiber0, scores_5classes_FCM_fiber0]
scores_classes_FCM_fiber0 = [float(np.average(score_classes_FCM_fiber0.get("acc").get("test"))) for score_classes_FCM_fiber0 in scores]

scores = [scores_2classes_FCM_fiber9, scores_3classes_FCM_fiber9, scores_4classes_FCM_fiber9, scores_5classes_FCM_fiber9]
scores_classes_FCM_fiber9 = [float(np.average(score_classes_FCM_fiber9.get("acc").get("test"))) for score_classes_FCM_fiber9 in scores]

classifier_FCM_df = pl.DataFrame(
        {"Classes": range(2, 6), "B2B": scores_classes_FCM_B2B, "Fiber @ 0dBm": scores_classes_FCM_fiber0, "Fiber @ 9dBm": scores_classes_FCM_fiber9}
    )

classifier_FCM_df

In [ ]:
scores = [scores_2classes_GKM_B2B, scores_3classes_GKM_B2B, scores_4classes_GKM_B2B, scores_5classes_GKM_B2B]
scores_classes_GKM_B2B = [float(np.average(score_classes_GKM_B2B.get("acc").get("test"))) for score_classes_GKM_B2B in scores]

scores = [scores_2classes_GKM_fiber0, scores_3classes_GKM_fiber0, scores_4classes_GKM_fiber0, scores_5classes_GKM_fiber0]
scores_classes_GKM_fiber0 = [float(np.average(score_classes_GKM_fiber0.get("acc").get("test"))) for score_classes_GKM_fiber0 in scores]

scores = [scores_2classes_GKM_fiber9, scores_3classes_GKM_fiber9, scores_4classes_GKM_fiber9, scores_5classes_GKM_fiber9]
scores_classes_GKM_fiber9 = [float(np.average(score_classes_GKM_fiber9.get("acc").get("test"))) for score_classes_GKM_fiber9 in scores]

classifier_GKM_df = pl.DataFrame(
        {"Classes": range(2, 6), "B2B": scores_classes_GKM_B2B, "Fiber @ 0dBm": scores_classes_GKM_fiber0, "Fiber @ 9dBm": scores_classes_GKM_fiber9}
    )

classifier_GKM_df